In [1]:
# things we need for NLP

import nltk
import tensorflow as tf
import random
import numpy as np
import pickle
import json

from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD


In [2]:
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('./data/intents.json', encoding = "utf-8").read())

In [3]:
print(intents)

{'intents': [{'tag': 'bienvenida', 'patterns': ['Hola', 'hay alguien hay', 'buenos días ', 'buen día', 'Hooooooola', 'qué tal'], 'responses': ['Hola', 'Buenos días', 'Buenos días \n ¡Qué gusto tenerte por acá!', 'Hola ¿en qué te puedo ayudar?']}, {'tag': 'confirmacion', 'patterns': ['Gracias', 'vale', 'entiendo'], 'responses': ['Por nada, ¿algo más en que puedo ayudarte?', 'A tu disposición, ¿alguna otra cosa?', 'Un placer. ¿qué más puedo hacer por ti?']}, {'tag': 'medicamentos', 'patterns': ['medicina', 'mis medicamentos, cuales son', 'cuál medicina me toca ahora', 'pastillas', 'jarabe', 'no sé cuándo reocger mi medicamento', 'vacuna'], 'responses': ['Ya reviso tu historial, un segundo', 'Ya vamos a revisar qué pasa. Un minuto, por favor']}, {'tag': 'estado_de_animo', 'patterns': ['hoy no me he sentido bien', 'me siento mejor', 'a ver qué tal me siento', 'un poco de malestar'], 'responses': ['exactamente, ¿qué tienes?', '¿qué te ha pasado?', '¿quieres conversar sobre esto?']}, {'tag':

In [4]:
words = []
classes = []
documents = []
ignore_words = ['?', ',', '!']

for intent in intents['intents']: 
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes: 
            classes.append(intent['tag'])
print(documents)


[(['Hola'], 'bienvenida'), (['hay', 'alguien', 'hay'], 'bienvenida'), (['buenos', 'días'], 'bienvenida'), (['buen', 'día'], 'bienvenida'), (['Hooooooola'], 'bienvenida'), (['qué', 'tal'], 'bienvenida'), (['Gracias'], 'confirmacion'), (['vale'], 'confirmacion'), (['entiendo'], 'confirmacion'), (['medicina'], 'medicamentos'), (['mis', 'medicamentos', ',', 'cuales', 'son'], 'medicamentos'), (['cuál', 'medicina', 'me', 'toca', 'ahora'], 'medicamentos'), (['pastillas'], 'medicamentos'), (['jarabe'], 'medicamentos'), (['no', 'sé', 'cuándo', 'reocger', 'mi', 'medicamento'], 'medicamentos'), (['vacuna'], 'medicamentos'), (['hoy', 'no', 'me', 'he', 'sentido', 'bien'], 'estado_de_animo'), (['me', 'siento', 'mejor'], 'estado_de_animo'), (['a', 'ver', 'qué', 'tal', 'me', 'siento'], 'estado_de_animo'), (['un', 'poco', 'de', 'malestar'], 'estado_de_animo'), (['¿cuándo', 'es', 'mi', 'próxima', 'cita', '?'], 'cita'), (['me', 'puedes', 'recordar', 'mi', 'próxima', 'cita'], 'cita'), (['no', 'recuerdo', 

In [5]:

words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))
print(len(documents), "documents")
print(len(classes), "labels \n", classes)
print(len(words), "unique stemmed words \n", words)

31 documents
6 labels 
 ['bienvenida', 'cita', 'confirmacion', 'despedida', 'estado_de_animo', 'medicamentos']
60 unique stemmed words 
 ['a', 'adios', 'ahora', 'alguien', 'bien', 'buen', 'buenos', 'bye', 'chao', 'cita', 'cuales', 'cuando', 'cuál', 'cuándo', 'de', 'desconecto', 'día', 'días', 'e', 'entiendo', 'gracias', 'hasta', 'hay', 'he', 'hola', 'hooooooola', 'hoy', 'jarabe', 'luego', 'malestar', 'me', 'medicamento', 'medicamentos', 'medicina', 'mejor', 'mi', 'médico', 'no', 'ocasión', 'otra', 'pastillas', 'poco', 'próxima', 'puedes', 'qué', 'recordar', 'recuerdo', 'reocger', 'sentido', 'siento', 'son', 'sé', 'tal', 'toca', 'un', 'vacuna', 'vale', 'vemos', 'ver', '¿cuándo']


In [6]:
pickle.dump(words, open('./data/words.pkl', 'wb'))
pickle.dump(classes, open('./data/classes.pkl', 'wb'))

In [7]:
#vectorización de los patrones para la DNN

training = []
output = []
output_empty = [0] * len(classes)

for doc in documents: 
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words] 
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])


In [8]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training, dtype="object")

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [9]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation = 'softmax'))

sgd = SGD(learning_rate= 0.01, decay = 1e-6, momentum=0.9, nesterov = True)

model.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics =['accuracy'])
hist = model.fit(np.array(train_x), np.array(train_y), epochs = 200, batch_size = 5, verbose = 0)

print("Listo, Calisto")

Listo, Calisto


In [10]:
model.save('./data/chatbot_model.h5', hist)